In [1]:
%load_ext autoreload
%autoreload 2
import multiprocessing
multiprocessing.set_start_method("spawn")
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [2]:
import argparse
import numpy as np
import torch
import os
from collections import Counter
from tqdm.auto import tqdm
import sys
from datetime import datetime
import sys
import os
from utils import (
    same_recons, get_topk_acc, get_layer_names,
    evaluate_performance, Logger, prepare_encoders, get_layer_names_new
)
from intervention_utils import intervension
from NeSymReS.Faithfulness.IterativePatching import load_dataset, initialize_model, prepare_inputs

In [3]:
import json
with open("../../circuit_configs.json", "r") as f:
    configs = json.load(f)

In [4]:
def select_topk_dataset(ranks, datapoints, top1=0, top2=0, top3=0, seed=42):
    import numpy as np

    np.random.seed(seed)

    ranks = np.array(ranks)
    datapoints = np.array(datapoints, dtype=object)

    idx_top1 = np.where(ranks == 1)[0]
    idx_top2 = np.where(ranks == 2)[0]
    idx_top3 = np.where(ranks == 3)[0]

    if len(idx_top1) < top1 or len(idx_top2) < top2 or len(idx_top3) < top3:
        raise ValueError("Not enough samples in one or more top-k categories.")

    selected_idx_1 = np.random.choice(idx_top1, top1, replace=False)
    selected_idx_2 = np.random.choice(idx_top2, top2, replace=False)
    selected_idx_3 = np.random.choice(idx_top3, top3, replace=False)

    all_selected_idx = np.concatenate([selected_idx_1, selected_idx_2, selected_idx_3])
    remaining_idx = np.setdiff1d(np.arange(len(ranks)), all_selected_idx)

    return all_selected_idx.tolist(), remaining_idx.tolist()

In [5]:
T1, T2, T3 = 73, 10, 17
BASE_PATH = "/home/arco/Downloads/Master/MscThesis/ExplainableDSR/data/Arco/CircuitFinding/"

In [ ]:
import glob
import numpy as np

intervention_class, model = initialize_model(base_path="../..")
full_range = set(range(0, 117))
operations = ["add", "cos", "exp", "log", "mul", "pow", "sin", "tan"]

for operation in operations:

    MAX_SAMPLES = 500
    
    print("\n\n-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-")
    print(f"Operation: {operation}")
    print("-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-")
    
    layer_names = get_layer_names_new()

    # Load precomputed results
    
    print("Loading datasets...")
    file_path = glob.glob(f"/home/arco/Downloads/Master/MscThesis/ExplainableDSR/data/Arco/CircuitFinding/dataset_{operation}_*.npz")
    datapoints = np.load(file_path[0], allow_pickle=True)["correct"]
    
    op = operation if operation != "log" else "ln"
    datapoints = same_recons(datapoints, target_operation=op)
    datapoints = [s for s in datapoints if s[0]["reconstructed_as"] == "correctly_predicted"]
    datapoints = np.array(datapoints, dtype=object)

    mean_patches = np.load("../../data/Arco/MeanPatching/mean_patching_10000.npy", allow_pickle=True).item()
    print("Dataset Loaded")
    
    print("Datapoints:", len(datapoints))
    Xs, ys, equations, eqs_untill_target = prepare_inputs(intervention_class, 
                                                          datapoints, 
                                                          200, 
                                                          max_samples=MAX_SAMPLES)
    
    # Baseline
    encoders = prepare_encoders(layer_names, [], mean_patches, "mean", model, Xs, ys)
    ranks, counter, logit_score = evaluate_performance(model,
                                                         intervention_class,
                                                         encoders,
                                                         equations,
                                                         eqs_untill_target,
                                                         operation,
                                                         return_activation=True)
    base_top1 = get_topk_acc(counter, 1)
    base_top2 = get_topk_acc(counter, 2)
    base_top3 = get_topk_acc(counter, 3)
    print(counter)
    print(f"model:              Top-1: {base_top1:.3f}, Top-2: {base_top2:.3f}, Top-3: {base_top3:.3f}, logit score: {logit_score.mean():.3f}")
    
    T1 = int(base_top1 * 100)
    T2 = int(base_top2 * 100) - T1
    T3 = 100 - T1 - T2
    
    train, test = select_topk_dataset(ranks, datapoints, top1=T1, top2=T2, top3=T3)
    
    encoders = prepare_encoders(layer_names, [], mean_patches, "mean", model, Xs[test], ys[test])
    ranks, counter, logit_score = evaluate_performance(model,
                                                         intervention_class,
                                                         encoders,
                                                         equations[test],
                                                         eqs_untill_target[test],
                                                         operation,
                                                         return_activation=True)
    base_top1 = get_topk_acc(counter, 1)
    base_top2 = get_topk_acc(counter, 2)
    base_top3 = get_topk_acc(counter, 3)
    print(f"model:              Top-1: {base_top1:.3f}, Top-2: {base_top2:.3f}, Top-3: {base_top3:.3f}, logit score: {logit_score.mean():.3f}")
    print("T1, T2, T3:", T1, T2, T3)

    save_path = f"{BASE_PATH}CD_TRAIN_{operation}_{T1}_{T2}_{T3}"
    np.save(save_path, datapoints[train])
    save_path = f"{BASE_PATH}CD_TEST_{operation}_{T1}_{T2}_{T3}"
    np.save(save_path, datapoints[test])

Running on GPU


-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-
Operation: log
-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-
Loading datasets...
Dataset Loaded
Datapoints: 309
Counter({3: 116, 2: 99, 1: 94})
model:              Top-1: 0.304, Top-2: 0.625, Top-3: 1.000, logit score: 0.311
model:              Top-1: 0.306, Top-2: 0.627, Top-3: 1.000, logit score: 0.310
T1, T2, T3: 30 32 38


-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-
Operation: add
-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-
Loading datasets...
Dataset Loaded
Datapoints: 500
Counter({1: 463, 3: 32, 2: 5})
model:              Top-1: 0.926, Top-2: 0.936, Top-3: 1.000, logit score: 0.922
model:              Top-1: 0.927, Top-2: 0.938, Top-3: 1.000, logit score: 0.924
T1, T2, T3: 92 1 7


-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-
Operation: cos
-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-=+=-
Loading datasets...
Dataset Loaded
Datapoints: 500
Counter({1: 473, 3: 15, 2: 12})
model:              Top-1: 0.946, Top-2: 0.970